# Prompting-Based Experiments

In [79]:
import re
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

import numpy as np
import pandas as pd

In [133]:
SHOTS = 3

In [80]:
sns.set_theme()
sns.set_style("whitegrid")
plt.style.use("alan.mplstyle")
ROOT = Path("../src/prompting/data")

In [147]:
meta_dyck_cot = json.load(open(ROOT / "meta-llama-s-3b-dyck.json"))
meta_dyck = json.load(open(ROOT / "meta-llama-3b-dyck-non.json"))

In [151]:
print(meta_dyck["predictions"][2])

<|end_of_text|><|end_of_text|><|end_of_text|><|begin_of_text|>Correctly complete the following Dyck language string.
Q: Complete the rest of the sequence, making sure that the parentheses are closed properly. Input: (){()
A: }

Correctly complete the following Dyck language string.
Q: Complete the rest of the sequence, making sure that the parentheses are closed properly. Input: {}()[
A: ]

Correctly complete the following Dyck language string.
Q: Complete the rest of the sequence, making sure that the parentheses are closed properly. Input: {([])
A: }

Correctly complete the following Dyck language string.
Q: Complete the rest of the sequence, making sure that the parentheses are closed properly. Input: ()(()
A: }

Correctly complete the following Dyck language string.
Q: Complete the rest of the sequence, making sure that the parentheses are closed properly. Input: {\}([{},])
A: }
<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|

In [143]:
def cot_parse(response):
    ptrn = re.compile("So the answer is [\}\)\]]")
    return re.findall(ptrn, response)

def parse(response):
    ptrn = re.compile("A: [\}\)\]]")
    return re.findall(ptrn, response)

def acc(response, target, parse_func):
    answers = parse_func(response)
    print(len(answers))
    if len(answers) <= 3:
        return False
    print(answers, target)
    return answers[SHOTS][-1] == target

In [145]:
correct_cot = 0
correct = 0
for i in range(len(meta_dyck_cot["predictions"])):
    pred, target = meta_dyck_cot["predictions"][i], meta_dyck_cot["targets"][i]
    correct_cot += acc(pred, target, cot_parse))

3
5
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )', 'So the answer is )'] }
3
3
3
5
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )', 'So the answer is )'] )
5
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )', 'So the answer is )'] }
3
4
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )'] ]
4
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )'] ]
3
3
4
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )'] ]
3
3
3
3
4
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )'] ]
4
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )'] }
4
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )'] ]
5
['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So th

In [128]:
print(cot_parse(meta_dyck_cot["predictions"][5]))

['So the answer is ]', 'So the answer is )', 'So the answer is }', 'So the answer is )', 'So the answer is )']


In [36]:
meta_dyck["predictions"][5].split("A:")[1]

' Let\'s think step by step.\nWe should process each input one by one and keep track of the stack configuration.\n0: empty stack\n1: [ ; stack: [\n2: { ; stack: [ {\n3: [ ; stack: [ { [\nNow, we have reached the end. The final stack is "[ { [".\nWe will need to pop out "[", "{", "[" one by one in that order.\nSo, we need "]", "}", "]". So the answer is ] } ].\n\nQ: Complete the rest of the sequence, making sure that the parentheses are closed properly. Input: < > ( ( [ [ ( { } ) [ < > ] ]\n'